In [1]:
import os
import pyspark
from pyspark.sql import SparkSession

In [2]:
sc = SparkSession \
        .builder \
        .master('spark://localhost:7077') \
        .appName("Spark") \
        .config("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.2_2.12:1.1.0") \
        .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
        .config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog") \
        .config("spark.sql.catalog.local.type", "hadoop") \
        .config("spark.sql.catalog.local.warehouse", "warehouse") \
        .config("spark.sql.defaultCatalog", "local") \
        .config("spark.sql.catalogImplementation", "in-memory") \
        .getOrCreate()

:: loading settings :: url = jar:file:/usr/local/lib/python3.8/dist-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.2_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-947591ca-da79-43f9-a814-e319c4811eae;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.2_2.12;1.1.0 in central
:: resolution report :: resolve 72ms :: artifacts dl 2ms
	:: modules in use:
	org.apache.iceberg#iceberg-spark-runtime-3.2_2.12;1.1.0 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submi

23/02/03 23:57:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
sc

In [6]:
!ls ../

agc-demo-1.ipynb  prepare_dataset.sh  README.md			      spark
old		  presto	      Run2012B_SingleMu-1000.parquet


In [8]:
df = sc.read.parquet("../Run2012B_SingleMu-1000.parquet")
df

DataFrame[run: int, luminosityBlock: bigint, event: bigint, MET: struct<pt:float,phi:float,sumet:float,significance:float,CovXX:float,CovXY:float,CovYY:float>, HLT: struct<IsoMu24_eta2p1:boolean,IsoMu24:boolean,IsoMu17_eta2p1_LooseIsoPFTau20:boolean>, PV: struct<npvs:int,x:float,y:float,z:float>, Muon: array<struct<pt:float,eta:float,phi:float,mass:float,charge:int,pfRelIso03_all:float,pfRelIso04_all:float,tightId:boolean,softId:boolean,dxy:float,dxyErr:float,dz:float,dzErr:float,jetIdx:int,genPartIdx:int>>, Electron: array<struct<pt:float,eta:float,phi:float,mass:float,charge:int,pfRelIso03_all:float,dxy:float,dxyErr:float,dz:float,dzErr:float,cutBasedId:boolean,pfId:boolean,jetIdx:int,genPartIdx:int>>, Photon: array<struct<pt:float,eta:float,phi:float,mass:float,charge:int,pfRelIso03_all:float,jetIdx:int,genPartIdx:int>>, Jet: array<struct<pt:float,eta:float,phi:float,mass:float,puId:boolean,btag:float>>, Tau: array<struct<pt:float,eta:float,phi:float,mass:float,charge:int,decayMode:

In [9]:
df = df[df['event']<62362200]

In [10]:
df.writeTo("iris_hep").partitionedBy('event').using("iceberg").create()